In [1]:
import gym
from gym.wrappers import Monitor
from scipy.stats import multivariate_normal
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import MultivariateNormal, Beta
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import json

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

env = gym.make('CarRacing-v0')
env = Monitor(env, './video', video_callable=lambda episode_id: episode_id%100==0, force=True)

d:\documents francis\université\session8\inf8225\projet\gym\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
class Gaussian(nn.Module):
    def __init__(self):
        super().__init__()
        
        

    def forward(self, mean_actions, std_actions, old_actions):
        # cov_mat = torch.diag(std_actions.mean(dim=0))
        distribution = Beta(mean_actions, std_actions)
        actions_with_exploration = distribution.sample()

        if old_actions is None:
            log_actions = distribution.log_prob(actions_with_exploration)
        else:
            log_actions = distribution.log_prob(old_actions)

        return actions_with_exploration, log_actions, distribution.entropy()

class CNN_Network(nn.Module):
    def __init__(self, output_size):
        super(CNN_Network, self).__init__()

        self.conv2d_0 = nn.Conv2d(1, 8, kernel_size=4, stride=2)
        self.relu_0 = nn.ReLU()

        self.conv2d_1 = nn.Conv2d(8, 16, kernel_size=3, stride=2)
        self.relu_1 = nn.ReLU()

        self.conv2d_2 = nn.Conv2d(16, 32, kernel_size=3, stride=2)
        self.relu_2 = nn.ReLU()

        self.conv2d_3 = nn.Conv2d(32, 64, kernel_size=3, stride=2)
        self.relu_3 = nn.ReLU()

        self.conv2d_4 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.relu_4 = nn.ReLU()

        self.conv2d_5 = nn.Conv2d(128, 256, kernel_size=3, stride=2)
        self.relu_5 = nn.ReLU()

        # self.action_linear_1 = nn.Linear(4608, 256)
        self.action_mean = nn.Linear(256, output_size)
        self.action_std = nn.Linear(256, output_size)
        self.mean_activation = nn.Softplus()
        self.std_activation = nn.Softplus()
        
        self.gaussian = Gaussian()


        # self.value_linear_1 = nn.Linear(4608, 256)
        # self.relu_value_1 = nn.LeakyReLU()
        self.value_linear_2 = nn.Linear(256, 1)

        for layer in [self.conv2d_0 , self.conv2d_1, self.conv2d_2, self.conv2d_3, self.conv2d_4, self.conv2d_5, self.action_mean, self.action_std,\
                      self.value_linear_2]:
            torch.nn.init.xavier_normal_(layer.weight)
            torch.nn.init.zeros_(layer.bias)

    def forward(self, x, old_actions=None):
        x = self.relu_0(self.conv2d_0(x))
        x = self.relu_1(self.conv2d_1(x))
        x = self.relu_2(self.conv2d_2(x))
        x = self.relu_3(self.conv2d_3(x))
        x = self.relu_4(self.conv2d_4(x))
        x = self.relu_5(self.conv2d_5(x))
        x = x.view(x.shape[0], -1)

        x_action_mean = self.mean_activation(self.action_mean(x))
        x_action_std = self.std_activation(self.action_std(x))
        actions, log_actions, entropy = self.gaussian(x_action_mean, x_action_std, old_actions)
        # x_action = self.activation_action1(x_action)

        # x_value = self.relu_value_1(self.value_linear_1(x))
        x_value = self.value_linear_2(x)
        return actions, log_actions, entropy, x_value

agent = CNN_Network(3).to(device)

In [3]:
print(list(list(agent.children())[-2].parameters()))

[]


In [4]:

# class OrnsteinUhlenbeckActionNoise():
#     def __init__(self, mu, sigma=0.3, theta=.10, dt=1e-2, x0=None):
#         self.theta = theta
#         self.mu = mu
#         self.sigma = sigma
#         self.dt = dt
#         self.x0 = x0
#         self.reset()

#     def __call__(self):
#         x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
#                 self.sigma * self.dt**(1/2) * torch.normal(mean=0.0, std=1.0, size=self.mu.shape, device=device)
#         self.x_prev = x
#         return x

#     def reset(self):
#         self.x_prev = self.x0 if self.x0 is not None else torch.zeros_like(self.mu, device=device)

#     def __repr__(self):
#         return 'OrnsteinUhlenbeckActionNoise(mu={}, sigma={})'.format(self.mu, self.sigma)

In [5]:
def to_torch(tensor):
    return torch.tensor(tensor.copy(), dtype=torch.float32, device=device)

def get_action(state):
    actions, log_actions, _, _ = agent(state)
    actions = actions[0]
    log_actions = log_actions[0]

    return  actions.cpu().numpy(), log_actions.cpu().numpy()

GAMMA = 0.99
def rollout(render, exploration_scale):

    state = env.reset() / 255.
    memory = []
    timesteps = 0
    done = False
    streak = 0
    while not done:
        
        action, log_action = get_action(to_torch(state).mean(dim=2).reshape(1, 1, state.shape[0], state.shape[1]))
        
        fixed_action = action.copy()*[2.,1.3,1.5] - [1.,0.3,0.5]
        next_state, reward, done, _ = env.step(fixed_action)

        # reward += 0.1*np.clip(action[1], 0, 1)
        next_state = next_state / 255.

        memory.append([state, action, reward, log_action])
        timesteps += 1
        state = next_state
            

    states, actions, rewards, log_actions = map(np.array, zip(*memory))

    discounted_rewards = np.zeros((len(rewards)))
    discount = 0
    # Discounts rewards in reverse
    for i in reversed(range(len(rewards))):

        # Discount fowards from the future for previous states
        discount = rewards[i] + discount*GAMMA 
        discounted_rewards[i] = discount

    return to_torch(states).mean(dim=3).unsqueeze(dim=1), to_torch(actions), to_torch(discounted_rewards).reshape(-1,1), to_torch(log_actions), timesteps, np.sum(rewards)

In [6]:
CLIP_EPSILON = 0.2
value_loss_fn = nn.SmoothL1Loss()

def get_log_probs_and_value(states, old_actions):
    actions, log_actions_new_policy, entropy, values = agent(states, old_actions)

    return actions, log_actions_new_policy, values, entropy

def compute_advantages(states, rewards):
    _, _, _, values = agent(states)

    advantages = rewards - values.detach()
    advantages = (advantages - advantages.mean()) / \
                (advantages.std() + 1e-8) 

    return advantages

def compute_losses(states, actions, rewards, log_actions, advantages):
    # Compute policy loss first
    # Compute ratios 
    new_actions, log_actions_new_policy, values, entropy = get_log_probs_and_value(states, actions)
    ratios = torch.exp(log_actions_new_policy - log_actions) 

    policy_loss = torch.min(ratios*advantages, torch.clip(ratios, 1 - CLIP_EPSILON, 1 + CLIP_EPSILON)*advantages)
    policy_loss = -torch.mean(policy_loss)

    # Compute value loss
    value_loss = value_loss_fn(rewards, values)

    # Compute entropy loss
    entropy_loss = -torch.mean(entropy)
    
    return policy_loss, value_loss, entropy_loss

In [7]:
# TODOS
# 1. new_actions / actions - > en log_probs (log_new_actions - log_actions)
# 2. Trouver des meilleurs manières pour explorer
# 3. tweak value_factor, GAMMA, exploration_factor, lr 
# 4. Tweak le modèle pytorch, activations


def train():

    n_time_steps = 2000000
    n_updates_per_episode = 5
    
    value_factor = 0.2
    entropy_factor = 0.01

    current_time_step = 0
    exploration_factor = 0.995

    agent_optimizer = optim.Adam(agent.parameters(), lr=0.0001)
    scores = []
    entropies = []
    while current_time_step < n_time_steps:
        # try:
        with torch.no_grad():
            agent.eval()
            states, actions, rewards, log_actions, timesteps, episode_score = rollout(len(scores)%5==0, exploration_factor**len(scores))
        # except:
        #     continue
        scores.append(episode_score)
        print(f"Current score: {scores[-1]}")
        print(f"Total timesteps: {current_time_step}")
        current_time_step += timesteps
        advantages = compute_advantages(states, rewards)

        agent.train()
        for _ in range(n_updates_per_episode):
            agent_optimizer.zero_grad()
            policy_loss, value_loss, entropy_loss = compute_losses(states, actions, rewards, log_actions, advantages)

            loss = policy_loss + value_factor*value_loss + entropy_factor*entropy_loss
            print(f'loss: {loss}')
            loss.backward()

            agent_optimizer.step()
            agent_optimizer.zero_grad()


    
    with open('results.json', 'w') as f:
        json.dump({"scores": scores}, f)
        
    torch.save(agent, 'agent.pt')



In [8]:
train()

Track generation: 1188..1489 -> 301-tiles track
Current score: -43.23333333333396
Total timesteps: 0
loss: 0.8160398602485657
loss: 0.8143383860588074
loss: 0.8126128911972046
loss: 0.8108115196228027
loss: 0.8089419007301331
Track generation: 1103..1390 -> 287-tiles track
Current score: -40.459440559441184
Total timesteps: 999
loss: 0.7435081601142883
loss: 0.7413901686668396
loss: 0.7391061186790466
loss: 0.7366225123405457
loss: 0.7339299917221069
Track generation: 1167..1463 -> 296-tiles track
Current score: -42.27288135593284
Total timesteps: 1998
loss: 0.794545590877533
loss: 0.7915090322494507
loss: 0.788148045539856
loss: 0.7844306826591492
loss: 0.7803208827972412
Track generation: 1101..1380 -> 279-tiles track
Current score: -38.74892086330983
Total timesteps: 2997
loss: 0.6916165947914124
loss: 0.686255156993866
loss: 0.6803322434425354
loss: 0.6737878322601318
loss: 0.6665197014808655
Track generation: 1239..1553 -> 314-tiles track
Current score: -42.39201277955332
Total ti

Error: Tried to reset environment which is not done. While the monitor is active for CarRacing-v0, you cannot call reset() unless the episode is over.